### Time Series Stuff

#### Basic model

In cross-sectional (single observations of a population), we have $n$ individual units of observation each observed once. In the basic time series model, we have $t$ observations of a single individual. Each value can be seen as being partly predicted by the previous value.

Thus, to structure a regression, you can create a "lagged" version of the dependent variable where each observation is partly explained by the previous one:
$$ \begin{aligned}
    y_t = \alpha y_{t-1} + \epsilon
\end{aligned} $$
For example, GDP this quarter is modellled as being explained by GDP last quarter.

This extends easily to multiple lags:
$$ \begin{aligned}
    y_t = \alpha y_{t-1} + \beta y_{t-2} + \epsilon
\end{aligned} $$

To do this, take the input array and set each value to the previous one in the case of one lag, or the $i - l$th value in the case of $l$ lags. The Pandas function [`shift()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.shift.html) will do this for you.

One thing to be aware of is that if you lag a column by a period, then you won't be able to use the last row as it won't have data for that lagged period. I think we are going to have maybe 3 years of daily data so a few days lags won't cost much, but if you want to add a lag of a month, you would lose thirty observations. ( _Note: this is at least this is how it works in Economics, maybe Data Science has invented magic ways around it, so this might be wrong_ ).

#### Time leakage

In the cross-sectional examples we did thus far, we did not need to make any assumptions about the availability of the training data for the model. However, for time series data, you can't use data that comes after a given period to predict that period - $y_t$ can rely only on information available in $y_t$. As such, it is probably best to structure the model so that the dependent variable is only dependent on data from previous time periods:
$$ \begin{aligned}
    y_t = \alpha y_{t-1} + \beta x_{t-1} + \gamma z_{t-1} + \epsilon 
\end{aligned} $$
Where, continuing to use our GDP example, the current quarter's GDP is dependent on the previous quarter's GDP, the previous quarter's inflation rate, $x_{t-1}$  and the previous quarter's employment rate, $z_{t-1}$. 

#### Controlling for seasonality

![Time Series Decomposition](timeSeriesDecomposition.png)

In general a time series can be composed of a number of concurrent series. These are usually considered a trend compoenent (the second panel), a seasonal component (the third panel) and the residual. We're interested in predicting the residual. As far as I understand from the questions I asked Joan in class, we won't need to worry about trends, he will give us data that is stationary (_i.e._ doesn't have a trend). We will need to do some controls for seasonality though, which is how the time of year or the day of the week affects the data generating process.

For example, continuing with the GDP example, we could expect that which quarter it is would affect the GDP in a given quarter, so we can generate the quarter from the data and use dummies:
$$ \begin{aligned}
    y_t = \alpha y_{t-1} + \delta_2 x_{t-1} + \delta_3 x_{t-1} + \delta_4 x_{t-1} + \epsilon 
\end{aligned} $$
Thus, we have dummies for quarters 2, 3 and 4, which allows us to control for seasonality. To generate the base control variable, you use code along the lines of the following:
``` python
    train_df['year'] = pd.to_datetime(train_df.my_date_column).dt.year
    train_df['quarter'] = pd.to_datetime(train_df.my_date_column).dt.quarter
    train_df['month'] = pd.to_datetime(train_df.my_date_column).dt.month
    train_df['week'] = pd.to_datetime(train_df.my_date_column).dt.week
    train_df['day_of_year'] = pd.to_datetime(train_df.my_date_column).dt.dayofyear
    train_df['day_of_month'] = pd.to_datetime(train_df.my_date_column).dt.day
```
and then generate dummies from these as you normally would.

Just in case the trend does come up, you can test the data (using an Augmented Dickey-Fuller test) to determine the correct number of lags to difference by, which leads to a rate parameter rather than the true parameter. In this case, you will lose all of your fixed effects. But, as I said, I'm pretty sure we won't need to do this.

#### Handling missing data

For cross-sectional data, because we had limited assumptions about the underlying structure, we were generally happy to drop rows that had missing data. Because, quite often, in time series, we only have a single dependent variable that is a regressor for subsequent periods we are not generally able to drop rows arbitrarily. As such, to handle missing data, I think it is better to drop columns or to use linear interpolation rather than to drop rows.

If there is a particular variable of interest that is only available after a certain period, it may be worthwhile to drop the rows before that period, but you have to drop all of them.

As discussed above, if you lag a variable by a certain number of periods $l$, then you need to drop the first $l$ periods of the series.

#### Splitting and sampling data
When we did the test train split or when we sampled the data in cross-sectional data, we needed to worry about stratification and class-balancing, but we were still primarily interested in random sampling. In time series, because of the serial structure, we are probably not going to do a random sample. Instead, the validation data will be the most recent 10-15\% of the sample as this is the closest to what you want to be able to predict. 

It seems advantageous to work through the entire process with a shorter sample to get a minimum viable product before working with the whole sample. In cross-sectional data, we used a stratified sample. Here, instead, we will just use the next most recent batch after the validation sample, _e.g,_ keep 10% of the data for validation, keep another 10-20% as a small sample to run through the whole process and then return and run the entire sample. (Just remember to burn the lagged data correctly both times.)




#### More than one time series
Issues arise if there is more than one dependent variable. To handle this, you can use joined regressions using models such as SEM, SUR or VAR which we haven't covered at all, and these generally require structural assumptions. As such I think we don't need to do them. Instead, it's probably easiest to work with separate regressions.

Handling multiple time series as independent variables is fine, just make sure to lag them so as to not make presumptions about release time (see the above section on time leakage).

### Process more generally

From everything we've done so far, it seems that what we're doing can be split into four main phases:
+ Preprocessing
+ Analysis
+ Post-processing
+ Visualisation
We've only really done two of them though, so I'll leave it at that...

#### Preprocessing
The preprocessing phase is where we take the data and make it ready to be used for analysis. It consists of the following stages:
+ Load the data
    + Read the file
    + Make a data frame
+ Check for nulls
    + Look at it by column 
    + Handle dependent variable separately
+ Generate derived controls
    + Generate lags, making sure to 
    + Generate time controls (e.g. quarters)
    + Generate dummies
+ Impute
    + For time series, use linear interpolation
    + It may make sense to impute categorical variables separately to quantitative variables more generally
    + Pickle the imputer
+ Scale the data
    + Separate the data into three column sets
        + Dependent variable(s)
        + Categorical column(s)
        + Quantitative column(s)
    + Scale the quantitative columns
    + Save the scaler to scale the validation data
    + Optionally, pickle the scaler
+ Transform
    + It may be necessary to transform the dependent variable or some of the independent variables (_e.g._ log of prices)
    + This will need to be undone later in post-processing
+ Save
    + This bit will take time, so it's probably a good idea to save the output of your data
    + You can also then share the file between you and your partner, to make sure you're getting the same results

#### Analysis
The analysis phase is where we do the main analysis of the different models to determine what we should use. It consists of the following stages:
+ Sample the data
    + In general, in order to get everything working, it seems useful to sample the data first
    + This is especially useful if you have an older laptop
+ Split the data
    + This may not be necessary, but if it is I think the validation data will be the most recent 10-15\% of the sample as this is the closest to what you want to be able to predict
+ Train the data
    + Use grid search on a set of of models (KNN, Decision Tree, Support Vector Machines)
    + Create ensembles of the best models
+ Validate the data
    + Display the scores of the best models
    + Make sure to detransform your variables if you transformed them.
